In [0]:
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
from matplotlib import pyplot as plt 
from sklearn import preprocessing
import statsmodels.formula.api as sm

url = 'https://raw.githubusercontent.com/AnoVando/TA1/master/TA1.csv'
TA1 = pd.read_csv(url)

In [0]:
TA1.columns = TA1.iloc[0]
TA1 = TA1.drop(TA1.index[[0]])
TA1.columns = ['Manufacturer', 'Screen_Size', 'Motion_Rate', 'Pixel', 'Overall_Rating', 'Video_Quality', 'Sound_Quality', 'Appearance',
              'Number_Features', 'Ease_Setup', 'Lifespan', 'Expert_Rating', 'Size_26', 'Size_32', 'Size_36', 'Size_40', 'Size_42',
              'Size_46', 'Size_49', 'Size_52', 'Size_55', 'Size_60', 'Motionrate_120', 'Motionrate_240', 'Motionrate_600',
              'Pixel_1080', "Selling_Price", 'Yearly_Units_Sold']

In [0]:
columns = ['Screen_Size', 'Motion_Rate', 'Pixel', 'Overall_Rating', 'Video_Quality', 'Sound_Quality', 'Appearance',
              'Number_Features', 'Ease_Setup', 'Lifespan', 'Expert_Rating', 'Size_26', 'Size_32', 'Size_36', 'Size_40', 'Size_42',
              'Size_46', 'Size_49', 'Size_52', 'Size_55', 'Size_60', 'Motionrate_120', 'Motionrate_240', 'Motionrate_600',
              'Pixel_1080', "Selling_Price", 'Yearly_Units_Sold']

TA1[columns] = TA1[columns].apply(pd.to_numeric)
TA1["Revenue"] = TA1["Selling_Price"] * TA1["Yearly_Units_Sold"]

In [14]:
results_ta1 = sm.ols(formula="Revenue ~ Video_Quality+Sound_Quality+Appearance+Number_Features+Lifespan+Ease_Setup \
                        +Expert_Rating+Size_26+Size_32+Size_36+Size_40+Size_42+Size_46+Size_49+Size_52 \
                        +Size_55+Size_60+Motionrate_120+Motionrate_240+Motionrate_600+Pixel_1080",
                data=TA1).fit()
print(results_ta1.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     1560.
Date:                Tue, 21 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:05:05   Log-Likelihood:                -10451.
No. Observations:                 703   AIC:                         2.095e+04
Df Residuals:                     681   BIC:                         2.105e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -7.819e+06   3.17e+05    -

## Bonus Credit - Regression Using Selling Price Differential

In [15]:
TA1['Segment']=TA1['Screen_Size'].astype(str)+'_'+TA1['Motion_Rate'].astype(str)+'_'+TA1['Pixel'].astype(str)
segment_price = TA1[['Segment', 'Selling_Price']].copy()
segment_price["Selling_Price"] = segment_price["Selling_Price"].apply(pd.to_numeric)
segment_price2 = segment_price.groupby('Segment', as_index=False)['Selling_Price'].mean()
segment_price2.columns = ["Segment", "Segment_Price"]
ta1_new = pd.merge(TA1, segment_price2, on='Segment', how='inner')
ta1_new["Price_Differential"] = ta1_new["Selling_Price"] - ta1_new["Segment_Price"]
ta1_new.head()

,Manufacturer,Screen_Size,Motion_Rate,Pixel,Overall_Rating,Video_Quality,Sound_Quality,Appearance,Number_Features,Ease_Setup,Lifespan,Expert_Rating,Size_26,Size_32,Size_36,Size_40,Size_42,Size_46,Size_49,Size_52,Size_55,Size_60,Motionrate_120,Motionrate_240,Motionrate_600,Pixel_1080,Selling_Price,Yearly_Units_Sold,Revenue,Segment,Segment_Price,Price_Differential
0,LG,19,60,1080,3.66,3.30,4.07,4.12,4.55,4.08,3.04,3.42,0,0,0,0,0,0,0,0,0,0,0,0,0,1,535,16364,8754740,19_60_1080,528.75,6.25
1,Panasonic,19,60,1080,4.21,4.43,3.31,3.47,3.71,4.19,4.67,4.02,0,0,0,0,0,0,0,0,0,0,0,0,0,1,550,16822,9252100,19_60_1080,528.75,21.25
2,Sony,19,60,1080,4.64,3.93,4.19,4.24,3.23,4.07,3.93,4.46,0,0,0,0,0,0,0,0,0,0,0,0,0,1,665,16183,10761695,19_60_1080,528.75,136.25
3,Samsung,19,60,1080,4.24,4.09,4.50,4.58,4.08,4.08,4.20,4.04,0,0,0,0,0,0,0,0,0,0,0,0,0,1,600,16893,10135800,19_60_1080,528.75,71.25
4,Philips,19,60,1080,3.58,3.32,3.48,3.78,2.63,3.22,3.56,3.30,0,0,0,0,0,0,0,0,0,0,0,0,0,1,490,15480,7585200,19_60_1080,528.75,-38.75


In [16]:
result_ta1_new = sm.ols(formula="Revenue ~ Video_Quality+Sound_Quality+Appearance+Number_Features+Lifespan+Ease_Setup \
                        +Expert_Rating+Size_26+Size_32+Size_36+Size_40+Size_42+Size_46+Size_49+Size_52 \
                        +Size_55+Size_60+Motionrate_120+Motionrate_240+Motionrate_600+Pixel_1080+Price_Differential",
                data=ta1_new).fit()
print(result_ta1_new.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     2525.
Date:                Tue, 21 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:05:13   Log-Likelihood:                -10268.
No. Observations:                 703   AIC:                         2.058e+04
Df Residuals:                     680   BIC:                         2.069e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           9.497e+05   4.74